## Dispositivos de E/S

Los dispositivos de E/S entran en escena porque todo ordenador necesita una interfaz con el mundo real. Este exterior puede ser como una PC donde el exterior es un ser humano entonces podriamos pensar monitor, mouse, etc. Pero no necesariamente todas las computadoras tienen monitor teclado etc. Por ejemplo una computadora orientada a predecir el clima necesitara sensores, etc. Los dispositivos de E/S son cualquier cosa que permita que variables del mundo exterior ingresen a la maquina, o al contrario algo que fue calculado por la maquina y que es expuesto al mundo exterior. Este es otro de los bloques de la arquitectura de Von Neumann. Como se conecta el dispositivo al ordenador, como lo ve el ordenador? la realidad es que los distintos dispositivos ocupan posiciones de memoria en el procesador. El procesador tiene un espacio de memoria dado por su bus de adress y no necesariamente tienen que estar conectadas en todas las direcciones celdas de memoria. Puede haber parte de ese espacio de memoria que este destinado a los dispositivos de E/S. Por ejemplo cuando uno presiona una tecla, en un teclado lo que hace el teclado es codificar todas las teclas con un numero, esos valores que se le asignan a las teclas se llaman scan codes y cada fabricante le pone scan codes propios a sus teclados. En definitiva cada vez que presionamos una tecla hay una posicion de memoria que tendra la ultima tecla que se apreto, el sistema leera las disintas teclas de memoria. Un monitor son miles de puntos, cada punto esta representado por RGB, graduando la intensidad se puede representar cualquier color. Cada punto de la pantalla entonces podrian ser 3 bytes que representan la intensidad. El procesador tendria que determinar la posicion del punto. Definitivamente no es optimo tener que estar consultando una y otra vez una posicion de memoria para ver si un evento ocurrio. El teclado PS/2 tenia dos posiciones de memoria asignadas al teclado, un byte de control representado por 7 bits que estaba en la posicion 0x64. Y en la posicion 0x60 habia un byte de datos. El bit data ready del registro de control indicaba si habia que leer algo o no, si esta en 0 el procesador dice que nadie apreto nada, si esta en 1, entonces se le indicaba al procesador que tenia que leer del registro de datos que es lo que se presiono, en el momento en que se lee el data ready se baja a 0. Si alguien aprieta la misma tecla devuelta se vuelve a prender el bit de data ready y el procesador vuelve a leer la siguiente tecla. A los efectos del procesador la informacion llega cuando se aprieta y cuando se suelta. Siempre que uno hace una combinacion entre dos dispositivos uno tiene que contemplar que llegue algo distinto, entocnes aparte del data ready hay dos bits mas parity error y el time out error. Si hay algun error en la comunicacion con el teclado el procesador podra darse cuenta que los datos son erroneos. Muchos dispositivos son bidireccionales, la funcion mas obvia es que el teclado le transmite a la computadora, pero por ejemplo cuando vamos a un programa y queremos prender la luz del teclado numerico obviamente es un comando que va del ordenador al teclado. En gral todos los dispositivos representan un conjunto de posiciones de memoria que la computadora puede leer o escribir para sacar o almancenar datos.

## Interrupciones

Es muy poco optimo que el procesador tenga que estar barriendo todos los data ready para ver si alguien tiene algo para transmitir, el SO deberia entrar en una rutina en la que debe chequear si paso algo. Para evitar esto los procesadores tienen un concepto de interrupcion. La idea basica detras de la interrupcion es, no me fijo en ningun dispositivo externo y los dispositivos externos me pueden avisar cuando tengan algo sin necesidad de que yo vaya byte por byte revisando si hay algun bit prendido para saber si hay algo. Todos los dispositivos de hoy en dia usan interrupciones. Escencialmente la interrupcion es un elemento de hardware que se agrega al procesador para ayudar a resolver el problema. Se agrega una entrada digita, una linea binaria al procesador que un dispositivo puede pulsar, y se modifica la logica del procesador, este normalmente obtenia la instruccion, la decodificaba, buscaba los parametros ejecutaba, guardaba los resultados y volvia a empezar, era como un ciclo permaemnte. Lo que vamos a hacer entonces es insertar algo en ese ciclo, cada vez que termina la instruccion y va a pasar a la siguiente, el procesador se fija en el estado de esa linea digital. Si la linea esta inactiva quiere decir que no hay nada que hacer y el ciclo continua, si cuando termina la instruccion ve que la linea esta activa entonces procede distinto. En primer lugar, coloca el valor del instruction pointer o program counter en el stack, pero hay un dato adicional, tambien guarda los flags en el stack y despues reemplaza el program counter con una direccion predefinida, esto es muy similar a un call, la diferencia es que el call no guarda los flags y esta operatoria si guarda los flags. Con lo cual el codigo sigue ejecutando en la neuva direccion, es la direccion de una rutina, esa rutina es la que va a chequear los dispositivos externos que esten en esa interrupcion. La direccion que tiene la rutina se llama vector de interrupcion. Y la rutina en si misma se llama rutina de servicio de interrupcion (ISR). Los procesadores mas chiquitos suelen tener una o dos interrupciones con dos vectores distintos, y las interrupciones muchas veces se pueden compartir. Que una interrupcion entre mientras se ejecuta la ISR puede llevar a resultadod indeseados. Por eso los procesadores tienen un bit en el registro Flags, que se denomina "Interrupt Mask". Se la puede poner en 1 con la instruccion STI o en 0 usando CLI. Mientras vale 1 el procesdor busca interrupciones al final de cada instruccion, cuando vale 0 sigue con la siguiente instruccion. Uno de los pasos realizados al detectar una interrupcion (justo despues de pasar los flags al stack), se baja a 0 el bit de interrupt mask. El codigo de la ISR ahora ejecuta sin que pueda entrar otra interrupcion. Al ejecutar iRET, cuando los Flags se restauran con la copa que se dejeo en el stack, el valor vuelve a 1 y puede entrar una nueva interrupcion. Los procesadores muy chiquitos suelen tener una o dos lineas de interrupcion, una IRQ que esta sujeta a la mascara y suelen tener otra interrupcion NMI que no tiene mascara y se usa para cosas criticas. Existe una interrupcion que permite llamar a otra intterrupcion. Estas interrupciones se denomina Software Interrputs y son la forma de acceder a las funciones de muchos SO de 16 o 32 bits. Para los Sistemas Operativos de 64 bits se usa un nuevo opcode denominado SYSCALL.